In [1]:
from moccasin import setup_notebook

setup_notebook()

In [3]:
from moccasin.config import get_or_initialize_config, get_active_network, Network

config = get_or_initialize_config()
config.set_active_network("arb-forked")
active_network: Network = get_active_network()
print(active_network.name)

arb-forked


In [5]:
import boa
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network, Network

STARTTING_ARB_BALANCE = int(1000e18)
STARTTING_USDC_BALANCE = int(100e6)  # usdc 6 decimals!


def _add_arb_balance():
    boa.env.set_balance(boa.env.eoa, STARTTING_ARB_BALANCE)


def _add_token_balance(usdc: ABIContract, arb: ABIContract):
    our_address = boa.env.eoa

    print(f"Starting balance of arb: {arb.balanceOf(our_address)}")
    with boa.env.prank(arb.owner()):
        arb.transferOwnership(our_address)
    arb.mint(our_address, STARTTING_ARB_BALANCE)
    print(f"Ending balance of ARB: {arb.balanceOf(our_address)}")

    print(f"Starting balance of USDC: {usdc.balanceOf(our_address)}")
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTTING_USDC_BALANCE)
    usdc.mint(our_address, STARTTING_USDC_BALANCE)
    print(f"Ending balance of USDC: {usdc.balanceOf(our_address)}")


def setup_script() -> Tuple[ABIContract, ABIContract, ABIContract, ABIContract]:
    print("Starting setup script...")
    active_network: Network = get_active_network()

    usdc = active_network.manifest_named("usdc")
    arb = active_network.manifest_named("arb")

    if active_network.is_local_or_forked_network():
        _add_arb_balance()
        _add_token_balance(usdc, arb)


def moccasin_main():
    setup_script()

moccasin_main()

Starting setup script...
Starting balance of arb: 0
Ending balance of ARB: 1000000000000000000000
Starting balance of USDC: 0
Ending balance of USDC: 100000000
